In [19]:
from parser import parse_character, parse_organisation
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ast
import re

# Парсинг

In [ ]:
# парсинг героев
df = pd.read_csv('data/characters.csv') # исходный файл, полученный со страниц по книгам
characters = list(df.character)
df_parsed = pd.DataFrame(parse_character(characters))
df_parsed.to_csv(f"data/characters_info.csv", index=False, encoding="utf-8")

Redirected from 'Cahir' → 'Cahir Mawr Dyffryn aep Ceallach'
Redirected from 'Vilgefortz' → 'Vilgefortz of Roggeveen'
Redirected from 'Yennefer' → 'Yennefer of Vengerberg'
Redirected from 'Addario Bach' → 'Addarion Bach'


In [79]:
# подготовка данных по организациям
df = pd.read_csv("data/characters_info.csv")

unique_aff = set()
df['affiliations'] = df['affiliations'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)
for item in df['affiliations']:
    if isinstance(item, list): 
        unique_aff.update(item)

print(f'Количество уникальных полей "affiliation": {len(unique_aff)}')

Количество уникальных полей "affiliation": 48


In [5]:
# парсинг организаций
organisations = list(unique_aff)
df_orgs = pd.DataFrame(parse_organisation(organisations))
df_orgs.drop(columns=['gender'], inplace=True)
df_orgs.to_csv(f"data/organisations_info.csv", index=False, encoding="utf-8")

Page 'Temerian Royal Council' parsed.
Page 'Redanian Secret Service' parsed.
Page 'Codringher and Fenn' parsed.
Page 'Chapter of the Gift and the Art' parsed.
Page 'Lytta Neyd' parsed.
Redirected from 'Skellen's gang' → 'Stefan Skellen's gang'
Page 'Stefan Skellen's gang' parsed.
Redirected from 'Wolf School' → 'School of the Wolf'
Page 'School of the Wolf' parsed.
Page 'Demavend III' parsed.
'King  Belohun of Kerack' error: The page you specified doesn't exist.
Redirected from 'Vilgefortz' → 'Vilgefortz of Roggeveen'
Page 'Vilgefortz of Roggeveen' parsed.
Page 'Syndicate' parsed.
Redirected from 'Kaedwenian Secret Service' → 'Kaedwenian intelligence'
Page 'Kaedwenian intelligence' parsed.
'Lyrian-Rivian Army White Queen's corps' error: The page you specified doesn't exist.
Page 'Order of the Flaming Rose' parsed.
Page 'Aelirenn' parsed.
Page 'Shani's Clinic' parsed.
Page 'Lodge of Sorceresses' parsed.
Page 'Belohun' parsed.
Page 'Oxenfurt Academy' parsed.
Page 'Big Four' parsed.
'King

# Обработка

## Organisations

In [67]:
def literal_clean(df, columns):
    for col in columns:
        df[col] = df[col].apply(lambda x: re.sub(r"[\[\]\'\"]", "", str(x)).strip() if pd.notna(x) else x)
    return df

In [68]:
df_orgs = pd.read_csv("data/organisations_info.csv")

In [69]:
print(f'{df_orgs.status.value_counts()}\n')

replace_dict = {
    'Extinct (': 'Extinct',
    'Temporarily disbanded by': 'Dissolved',  
    'Disbanded (?)': 'Dissolved',
    'Unknown':  None
}

df_orgs['status'] = df_orgs['status'].replace(replace_dict)

print(df_orgs.status.value_counts())

status
Extinct                     6
Extant                      3
Dissolved                   2
Temporarily disbanded by    1
Unknown                     1
Extinct (                   1
Disbanded (?)               1
Name: count, dtype: int64

status
Extinct      7
Dissolved    4
Extant       3
Name: count, dtype: int64


In [70]:
df_orgs = df_orgs[df_orgs['organisation'] != "Nilfgaardian Secret Service"] # дубль
df_orgs = df_orgs[df_orgs['organisation'] != "University of Oxenfurt"] # дубль

# очистка строк
df_orgs = literal_clean(df_orgs, ['type', 'purpose'])

In [71]:
print(f'{df_orgs.type.value_counts()}\n')

replace_dict = {
    'Secret Service': 'Secret service',
    'Crime organization': 'Criminal organization',  
    'Detective agency Law firm': 'Detective agency and Law firm',
    'nan':  None
}

df_orgs['type'] = df_orgs['type'].replace(replace_dict)
df_orgs = df_orgs[~df_orgs['type'].isin(['Castle', 'Building', 'School', 'Magical university', 'Bathhouse'])] # не организации
print(f'{df_orgs.type.value_counts()}')


type
Witcher school               4
Secret service               3
Inner circle of mages        2
Mage organization            2
Detective agency Law firm    1
Gang                         1
Crime organization           1
Military religious order     1
Building                     1
School                       1
Criminal organization        1
Secret Service               1
rag-tag company              1
Castle                       1
Magical university           1
Mercenary group              1
Bathhouse                    1
Name: count, dtype: int64

type
Secret service                   4
Witcher school                   4
Criminal organization            2
Inner circle of mages            2
Mage organization                2
Detective agency and Law firm    1
Gang                             1
Military religious order         1
rag-tag company                  1
Mercenary group                  1
Name: count, dtype: int64


In [72]:
df_orgs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 0 to 27
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   organisation   21 non-null     object
 1   full name      21 non-null     object
 2   status         14 non-null     object
 3   area served    13 non-null     object
 4   country        10 non-null     object
 5   type           19 non-null     object
 6   purpose        13 non-null     object
 7   founder        10 non-null     object
 8   leader         16 non-null     object
 9   members        8 non-null      object
 10  headquarters   13 non-null     object
 11  appears_books  17 non-null     object
 12  appears_games  14 non-null     object
dtypes: object(13)
memory usage: 2.3+ KB


In [74]:
df_orgs.to_csv(f"data/organisations_info_cleaned.csv", index=False, encoding="utf-8")

## Characters

In [75]:
df = pd.read_csv("data/characters_info.csv")
df.info()
df.drop(columns=['born'], axis=1, inplace=True)
df.dropna(subset=['gender'], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   character      64 non-null     object 
 1   full name      64 non-null     object 
 2   born           10 non-null     float64
 3   nationality    34 non-null     object 
 4   hair_color     41 non-null     object 
 5   eye_color      31 non-null     object 
 6   race           60 non-null     object 
 7   gender         62 non-null     object 
 8   status         29 non-null     object 
 9   titles         22 non-null     object 
 10  profession     52 non-null     object 
 11  affiliations   44 non-null     object 
 12  abilities      30 non-null     object 
 13  appears_books  64 non-null     object 
 14  appears_games  38 non-null     object 
dtypes: float64(1), object(14)
memory usage: 7.6+ KB


In [76]:
print(df.hair_color.value_counts())

replace_dict = {
    'Ashen grey': 'Ashen-gray',
    'Greyish white,': 'Greyish white'
}
df['hair_color'] = df['hair_color'].replace(replace_dict)

hair_color
Black                                    7
Grey                                     5
Brown                                    4
Dark                                     3
White                                    3
Blonde                                   2
Chestnut                                 2
Blond                                    1
Ashen grey                               1
Ashen-gray                               1
Greyish white,                           1
Black, later white                       1
Milk-white                               1
Red                                      1
Bald                                     1
Dark red                                 1
Straw blonde                             1
Dark gold                                1
Fox red                                  1
Flaxen                                   1
Ashen grey with snow-white highlights    1
Fair                                     1
Name: count, dtype: int64


In [77]:
df = literal_clean(df, ['titles', 'abilities', 'profession']) # очистка строк
df['isAlive'] = df['status'] != 'Deceased' # TF 

In [78]:
df.to_csv(f"data/characters_info_cleaned.csv", index=False, encoding="utf-8")